In [1]:
import pandas as pd   
import json
import requests

In [8]:
main()

In [2]:
def intro():
    header = { 'X-Application' : "",  'X-Authentication' : '', 'content-type' : 'application/json' }
    url="https://api.betfair.com/betting/json-rpc"
    #here you'd need to input your personal Application Key and Session Token, bound to your betfair account. 
    #Appkey does not change, however token is generated daily?/every time you log in. You need to be logged in to beftair.
    appkey = 'abc'
    token = 'def'
    header['X-Application'] = appkey
    header['X-Authentication'] = token
    return url, header

In [3]:
def get_matches(league_code,days,url,header):     
    jsonrpc_req={"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listEvents", "params": {"filter":{"eventTypeIds" : [1], "competitionIds": ''}}} 
    jsonrpc_req['params']['filter']['competitionIds'] = [league_code]
    jsonrpc_req = json.dumps(jsonrpc_req)
    response = requests.post(url, data=jsonrpc_req, headers=header)
    
    data1 = json.dumps(json.loads(response.text), indent=3)
    data2= json.loads(response.text)
    data3 = data2['result']

    TeamsPlaying = []
    MatchID = []
    TeamHome = []
    TeamAway = []

    from datetime import datetime, timedelta
    for index in data3:
        day = index['event']['openDate'][0:10]
        day_object = datetime.strptime(day, "%Y-%m-%d")
        today_p3 = datetime.today() + + timedelta(days)
        #this is needed to exlude the bet for the whole EPL winner
        if " v " in index['event']['name']:
            if day_object < today_p3:
                #if day_object > today_p1:
                TeamsPlaying.append(index['event']['name'])
                MatchID.append(index['event']['id'])
                TeamsSplit = index['event']['name'].split(' v ')
                TeamHome.append(TeamsSplit[0].strip())
                TeamAway.append(TeamsSplit[1].strip())

    col_name = ['Home','Away','MatchID']
    df_matches = pd.DataFrame([TeamHome,TeamAway,MatchID],index=col_name)
    df_matches = df_matches.T
    return df_matches

In [4]:
def marketonly_request(market,league,MatchID,url,header):    
    jsonrpc_req_mID = {"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketCatalogue", "params": {"filter":{"eventIds": '' },"maxResults": "700","marketProjection": ["COMPETITION","EVENT"]}}
    jsonrpc_req_mID['params']['filter']['eventIds'] = MatchID
    jsonrpc_req_mID = json.dumps(jsonrpc_req_mID)
    response_mID = requests.post(url, data=jsonrpc_req_mID, headers=header)

    data_m_1 = json.dumps(json.loads(response_mID.text), indent=3)
    data_m_2= json.loads(response_mID.text)
    data_m_3 = data_m_2['result']
    
    bets_name = []
    bets_id = []
    team_play = []
    home_play = []
    away_play = []
    league_table = []

    bets_needed = [market]
    league_name = [league]
    for index in data_m_3:
        if index['competition']['name'] in league_name:
            if index['marketName'] in bets_needed:
                    team_play.append(index['event']['name'])
                    bets_name.append(index['marketName'])
                    bets_id.append(index['marketId'])
                    league_table.append(index['competition']['name'])
                    TeamsSplit = index['event']['name'].split(' v ')
                    home_play.append(TeamsSplit[0].strip())
                    away_play.append(TeamsSplit[1].strip())
    
    col_markets = ['Home','Away','BetID','BetName','League']
    df_markets =  pd.DataFrame([home_play,away_play,bets_id,bets_name,league_table],index=col_markets)       
    df_markets = df_markets.T
    return df_markets

In [5]:
def bets_all_request(market,league,bets_id,url,header):    
    jsonrpc_req_bID = {"jsonrpc": "2.0", "method": "SportsAPING/v1.0/listMarketBook", "params": {"marketIds": '' ,"priceProjection": {"priceData": ["EX_BEST_OFFERS", "EX_TRADED"],"virtualise": "true"} }}
    jsonrpc_req_bID['params']['marketIds'] = bets_id
    jsonrpc_req_bID = json.dumps(jsonrpc_req_bID)
    response_bID = requests.post(url, data=jsonrpc_req_bID, headers=header)

    data_b_1 = json.dumps(json.loads(response_bID.text), indent=3)
    data_b_2= json.loads(response_bID.text)
    data_b_3 = data_b_2['result']
    data_b_3
    
    bets2way = ['Over/Under 0.5 Goals','Over/Under 1.5 Goals','Over/Under 2.5 Goals','Over/Under 3.5 Goals','Over/Under 4.5 Goals','Draw no Bet','Both teams to Score?']
    bets3way = ['Match Odds','Double Chance']
    
    bet_one_price = []
    bet_one_id_check = []
    bet_two_price = []
    bet_two_id_check = []

    for index in data_b_3:
        try:
            bet_one_price.append(index['runners'][0]['ex']['availableToBack'][0]['price'])
            bet_one_id_check.append(index['marketId'])
        except IndexError:
            pass

    for index in data_b_3:
        try:
            bet_two_price.append(index['runners'][1]['ex']['availableToBack'][0]['price'])
            bet_two_id_check.append(index['marketId'])
        except IndexError:
            pass
        
    if market in bets2way:
            col_prices = ['BetID','Over_price','Under_price']
            df_prices = pd.DataFrame([bet_one_id_check,bet_two_price, bet_one_price],index=col_prices)

        
    if market in bets3way:
        bet_x_price = []
        bet_x_id_check = []
        for index in data_b_3:
            try:
                bet_x_price.append(index['runners'][2]['ex']['availableToBack'][0]['price'])
                bet_x_id_check.append(index['marketId'])
            except IndexError:
                pass
        col_prices = ['BetID','home_win','away_win','draw']
        df_prices = pd.DataFrame([bet_one_id_check,bet_one_price, bet_two_price, bet_x_price],index=col_prices)
    
    df_prices = df_prices.T       
    return df_prices

In [6]:
def one_league(league,days):       
    if league == 'EPL':
        league_id = 10932509
        league_name = 'English Premier League'
    elif league =='Bundesliga':
        league_id = 59
        league_name = 'German Bundesliga'
    elif league == 'La_liga':
        league_id = 117
        league_name = 'Spanish La Liga'
    elif league == 'Serie_A':
        league_id = 81
        league_name = 'Italian Serie A'
    elif league  == 'Ligue_1':
        league_id = 55
        league_name = 'French Ligue 1'
        
    url = intro()[0]
    header = intro()[1]  
    
    df_matches = get_matches(league_id,days,url,header)
    MatchID =  df_matches.MatchID.tolist()
    
    bets_overs = ['Over/Under 0.5 Goals','Over/Under 1.5 Goals','Over/Under 2.5 Goals','Over/Under 3.5 Goals','Over/Under 4.5 Goals','Both teams to Score?']
    all_markets = ['Match Odds','Double Chance','Over/Under 0.5 Goals','Over/Under 1.5 Goals','Over/Under 2.5 Goals','Over/Under 3.5 Goals','Over/Under 4.5 Goals','Draw no Bet','Both teams to Score?']
    
    df_odds_overs_appended = []
    
    if "Empty DataFrame" not in str(df_matches):
        for elem in all_markets:
            df_markets_overs = marketonly_request(elem,league_name,MatchID,url,header)
            bets_id = df_markets_overs.BetID.tolist()
            df_prices_overs = bets_all_request(elem,league_name,bets_id,url,header)
            if elem == 'Both teams to Score?':   
                temp2 = df_prices_overs.Over_price
                df_prices_overs.Over_price = df_prices_overs.Under_price
                df_prices_overs.Under_price = temp2
            if elem in bets_overs:
                temp = pd.merge(df_markets_overs, df_prices_overs, on='BetID', how='outer')
                df_odds_overs_appended.append(temp)
            elif elem == 'Match Odds':
                df_odds_match = pd.merge(df_markets_overs, df_prices_overs, on='BetID', how='outer')
            elif elem == 'Draw no Bet':
                df_odds_dnb = pd.merge(df_markets_overs, df_prices_overs, on='BetID', how='outer')
                df_odds_dnb = df_odds_dnb.rename(columns={'Over_price':'Home_dnb','Under_price':'Away_dnb'})
                temp = df_odds_dnb.Home_dnb
                df_odds_dnb.Home_dnb = df_odds_dnb.Away_dnb
                df_odds_dnb.Away_dnb = temp
                df_odds_dnb = df_odds_dnb [['Home','Away','BetID','BetName','League','Home_dnb','Away_dnb']]
            elif elem == 'Double Chance':
                df_odds_dch = pd.merge(df_markets_overs, df_prices_overs, on='BetID', how='outer')            
                df_odds_dch = df_odds_dch.rename(columns={'home_win':'Home&draw','away_win':'Away&draw','draw':'Home&Away'})
                df_odds_dch = df_odds_dch[['Home','Away','BetID','BetName','League','Home&draw','Away&draw','Home&Away']]
        df_odds_overs = pd.concat(df_odds_overs_appended)
        
        return df_odds_overs, df_odds_match, df_odds_dnb, df_odds_dch

In [7]:
def main():   
    days = 1
    
    df_all = pd.DataFrame()
    df_all_match = pd.DataFrame()
    df_all_dnb = pd.DataFrame()
    df_all_dch = pd.DataFrame()
        
    all_leagues = ['EPL','Ligue_1','Bundesliga','La_liga','Serie_A']
    for league in all_leagues:
        if one_league(league,days) is not None:
            temp = one_league(league,days)
            df_all = pd.concat([df_all, temp[0]])
            df_all_match = pd.concat([df_all_match, temp[1]])
            df_all_dnb = pd.concat([df_all_dnb, temp[2]])
            df_all_dch = pd.concat([df_all_dch, temp[3]])
    
    df_all.sort_values(['League', 'BetName'], ascending=[True, True])
    df_all_match.sort_values(['League', 'BetName'], ascending=[True, True])   
    df_all_dnb.sort_values(['League', 'BetName'], ascending=[True, True])   
    df_all_dch.sort_values(['League', 'BetName'], ascending=[True, True])   
    
    path =  r'C:xxx/betfair_prices_big5.xlsx'
    path2 =  r'C:xxx/betfair_prices_big5_match.xlsx'
    with pd.ExcelWriter(path) as writer:
        df_all.to_excel(writer, sheet_name="AllData")
    with pd.ExcelWriter(path2) as writer:
        df_all_match.to_excel(writer, sheet_name="MatchOdds")
        df_all_dnb.to_excel(writer, sheet_name="DnB")
        df_all_dch.to_excel(writer, sheet_name="DCh")